In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import re

## Cleaning the data for the lines and their stations

In [60]:
df = pd.read_csv('../data/NBT20FRI-new.csv')

In [61]:
df

,Line,Dir,From Station,To Station
0,Bakerloo,NB,Elephant & Castle LU,Lambeth North
1,Bakerloo,NB,Lambeth North,Waterloo LU
2,Bakerloo,NB,Waterloo LU,Embankment
3,Bakerloo,NB,Embankment,Charing Cross LU
4,Bakerloo,NB,Charing Cross LU,Piccadilly Circus
...,...,...,...,...
1154,Victoria,SB,Pimlico,Vauxhall LU
1155,Victoria,SB,Vauxhall LU,Stockwell
1156,Victoria,SB,Stockwell,Brixton LU
1157,Waterloo & City,EB,Waterloo LU,Bank


In [62]:
def clean_station_name(name):
        # Remove suffixes like LU, LO, NR, TfL, EL
        name = re.sub(r'\s+(LU|LO|NR|TfL|EL)$', '', name)
        # Remove suffixes in brackets
        name = re.sub(r'\s+\([^)]+\)$', '', name)
        return name.strip()

def clean_tube_data(df):
    # Create a copy of the DataFrame to avoid SettingWithCopyWarning
    df = df.copy()
    
    # Choose one direction per line
    df = df.drop_duplicates(subset=['Line', 'From Station'])
    
    # Clean station names
    df['From Station'] = df['From Station'].apply(clean_station_name)
    df['To Station'] = df['To Station'].apply(clean_station_name)
    
    # Create a new dataframe with Line and Station Name
    from_stations = df[['Line', 'From Station']].rename(columns={'From Station': 'Station Name'})
    to_stations = df[['Line', 'To Station']].rename(columns={'To Station': 'Station Name'})
    stations = pd.concat([from_stations, to_stations], ignore_index=True)
    
    # Remove duplicates and sort
    stations = stations.drop_duplicates().sort_values(['Line', 'Station Name'])
    
    return stations.reset_index(drop=True)

In [63]:
clean_stations = clean_tube_data(df)
clean_stations

,Line,Station Name
0,Bakerloo,Baker Street
1,Bakerloo,Charing Cross
2,Bakerloo,Edgware Road
3,Bakerloo,Elephant & Castle
4,Bakerloo,Embankment
...,...,...
590,Victoria,Victoria
591,Victoria,Walthamstow Central
592,Victoria,Warren Street
593,Waterloo & City,Bank


In [64]:
clean_stations.to_csv('../data/station-with-no-coordinates.csv', sep=',', encoding='utf-8', index=False, header=True)

### Cleaning dataset from 2022

In [65]:
df_22 = pd.read_csv('../data/2022-TFL-Links.csv', header=2)
df_22.head(10)

,Link,Line,Dir,Order,From NLC,From ASC,From Station,To NLC,To ASC,To Station,...,0245-0300,0300-0315,0315-0330,0330-0345,0345-0400,0400-0415,0415-0430,0430-0445,0445-0500,Unnamed: 113
0,ELEu_BAK_NB>LAMu_BAK_NB@BAK,Bakerloo,NB,1,570,ELEu,Elephant & Castle LU,628,LAMu,Lambeth North,...,0,0,0,0,0,0,0,0,0,NaN
1,LAMu_BAK_NB>WLOu_BAK_NB@BAK,Bakerloo,NB,2,628,LAMu,Lambeth North,747,WLOu,Waterloo LU,...,0,0,0,0,0,0,0,0,0,NaN
2,WLOu_BAK_NB>EMBu_BAK_NB@BAK,Bakerloo,NB,3,747,WLOu,Waterloo LU,542,EMBu,Embankment,...,0,0,0,0,0,0,0,0,0,NaN
3,EMBu_BAK_NB>CHXu_BAK_NB@BAK,Bakerloo,NB,4,542,EMBu,Embankment,718,CHXu,Charing Cross LU,...,0,0,0,0,0,0,0,0,0,NaN
4,CHXu_BAK_NB>PICu_BAK_NB@BAK,Bakerloo,NB,5,718,CHXu,Charing Cross LU,674,PICu,Piccadilly Circus,...,0,0,0,0,0,0,0,0,0,NaN
5,PICu_BAK_NB>OXCu_BAK_NB@BAK,Bakerloo,NB,6,674,PICu,Piccadilly Circus,669,OXCu,Oxford Circus,...,0,0,0,0,0,0,0,0,0,NaN
6,OXCu_BAK_NB>RPKu_BAK_NB@BAK,Bakerloo,NB,7,669,OXCu,Oxford Circus,685,RPKu,Regent's Park,...,0,0,0,0,0,0,0,0,0,NaN
7,RPKu_BAK_NB>BSTu_BAK_NB@BAK,Bakerloo,NB,8,685,RPKu,Regent's Park,511,BSTu,Baker Street,...,0,0,0,0,0,0,0,0,0,NaN
8,BSTu_BAK_NB>MYBu_BAK_NB@BAK,Bakerloo,NB,9,511,BSTu,Baker Street,641,MYBu,Marylebone LU,...,0,0,0,0,0,0,0,0,0,NaN
9,MYBu_BAK_NB>ERBu_BAK_NB@BAK,Bakerloo,NB,10,641,MYBu,Marylebone LU,774,ERBu,Edgware Road (Bak),...,0,0,0,0,0,0,0,0,0,NaN


In [66]:
stations22 = clean_tube_data(df_22)
stations22.head(10)

,Line,Station Name
0,Bakerloo,Baker Street
1,Bakerloo,Charing Cross
2,Bakerloo,Edgware Road
3,Bakerloo,Elephant & Castle
4,Bakerloo,Embankment
5,Bakerloo,Harlesden
6,Bakerloo,Harrow & Wealdstone
7,Bakerloo,Kensal Green
8,Bakerloo,Kenton
9,Bakerloo,Kilburn Park


In [67]:
stations22.to_csv('../data/2022-tfl-stations.csv', sep=',', encoding='utf-8', index=False, header=True)

### Creating a seperate dataset for the links 2022

In [68]:
df_22 = df_22[['Line', 'From Station', 'To Station']]

In [69]:
df_22['From Station'] = df_22['From Station'].apply(clean_station_name)
df_22['To Station'] = df_22['To Station'].apply(clean_station_name)

In [70]:
df_22.head(10)

,Line,From Station,To Station
0,Bakerloo,Elephant & Castle,Lambeth North
1,Bakerloo,Lambeth North,Waterloo
2,Bakerloo,Waterloo,Embankment
3,Bakerloo,Embankment,Charing Cross
4,Bakerloo,Charing Cross,Piccadilly Circus
5,Bakerloo,Piccadilly Circus,Oxford Circus
6,Bakerloo,Oxford Circus,Regent's Park
7,Bakerloo,Regent's Park,Baker Street
8,Bakerloo,Baker Street,Marylebone
9,Bakerloo,Marylebone,Edgware Road


In [71]:
df_22['Line'].unique()

array(['Bakerloo', 'Central', 'District', 'DLR', 'Elizabeth Line',
       'H&C and Circle', 'Jubilee', 'LO East London',
       'LO Gospel Oak-Barking', 'LO North London', 'LO Romford–Upminster',
       'LO Watford-Euston', 'LO West Anglia', 'London Trams',
       'Metropolitan', 'Northern', 'Piccadilly', 'Victoria',
       'Waterloo & City'], dtype=object)

### Modifying Lines

In [73]:
def consolidate_overground(df):
    # List of London Overground lines
    lo_lines = ['LO East London', 'LO Gospel Oak-Barking', 'LO North London', 
                'LO Romford–Upminster', 'LO Watford-Euston', 'LO West Anglia']
    
    # Replace all LO lines with 'Overground'
    df['Line'] = df['Line'].replace(lo_lines, 'Overground')
    
    # Reset the index
    df.reset_index(drop=True, inplace=True)
    
    return df

In [74]:
df_22 = consolidate_overground(df_22)

In [75]:
df_22['Line'].unique()

array(['Bakerloo', 'Central', 'District', 'DLR', 'Elizabeth Line',
       'H&C and Circle', 'Jubilee', 'Overground', 'London Trams',
       'Metropolitan', 'Northern', 'Piccadilly', 'Victoria',
       'Waterloo & City'], dtype=object)

In [76]:
df_22.to_csv('../data/2022-tfl-edges.csv', sep=',', encoding='utf-8', index=False, header=True)